# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from glob import glob
import os

from pyopia.classify import Classify
import exampledata
import pyopia.io
from pyopia.pipeline import Pipeline
import pyopia.instrument.silcam
import pyopia.process
import pyopia.background

In [3]:
filename = exampledata.get_example_silc_image()

files = glob('raw_data/*.silc') # relies on having a list of raw files - not currently auto-obtained

In [4]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

os.makedirs('proc', exist_ok=True)

datafile_hdf = 'proc/test'
model_path = exampledata.get_example_model()
threshold = 0.85

average_window = 10  # number of images to use as background
bgfiles = files[:average_window]  # provide a list of background files used to create the static background

steps = {'classifier': Classify(model_path=model_path),
         'create background': pyopia.background.CreateBackground(bgfiles,
                                                                 pyopia.instrument.silcam.load_image),
         'load': pyopia.instrument.silcam.SilCamLoad(),
         'correct background': pyopia.background.CorrectBackgroundAccurate(),  # pyopia.background.shift_bgstack_accurate
         'imageprep': pyopia.instrument.silcam.ImagePrep(image_level='imc'),
         'segmentation': pyopia.process.Segment(threshold=threshold),
         'statextract': pyopia.process.CalculateStats(export_outputpath='export'),
         'output': pyopia.io.StatsH5(datafile_hdf)}

processing_pipeline = Pipeline(steps,
                               initial_steps=['classifier', 'create background'])

Initialising pipeline
  Running <pyopia.classify.Classify object at 0x00000270175C3760>
  Running <pyopia.background.CreateBackground object at 0x00000270323ACFD0>
Pipeline ready with these data:  ['cl', 'bgstack', 'imbg']


In [5]:
# If you want to load a pre-defined set of steps for silcam analysis, you can do it like this:
from pyopia.instrument.silcam import silcam_steps
default_steps, default_initial_steps = silcam_steps(model_path, threshold, datafile_hdf)
# then you would initialise the pipeline like this:
# processing_pipeline = Pipeline(default_steps, initial_steps=default_initial_steps)

In [6]:
processing_pipeline = Pipeline(steps, initial_steps=['classifier', 'create background'])
for filename in files:
    stats = processing_pipeline.run(filename)

Initialising pipeline
  Running <pyopia.classify.Classify object at 0x00000270175C3760>
  Running <pyopia.background.CreateBackground object at 0x00000270323ACFD0>
Pipeline ready with these data:  ['cl', 'bgstack', 'imbg']
calling:  <class 'pyopia.instrument.silcam.SilCamLoad'>  with:  ['cl', 'bgstack', 'imbg', 'filename', 'steps_string']
calling:  <class 'pyopia.background.CorrectBackgroundAccurate'>  with:  ['cl', 'bgstack', 'imbg', 'filename', 'steps_string', 'timestamp', 'imraw']
calling:  <class 'pyopia.instrument.silcam.ImagePrep'>  with:  ['cl', 'bgstack', 'imbg', 'filename', 'steps_string', 'timestamp', 'imraw', 'imc']
calling:  <class 'pyopia.process.Segment'>  with:  ['cl', 'bgstack', 'imbg', 'filename', 'steps_string', 'timestamp', 'imraw', 'imc', 'imref']
segment
clean
calling:  <class 'pyopia.process.CalculateStats'>  with:  ['cl', 'bgstack', 'imbg', 'filename', 'steps_string', 'timestamp', 'imraw', 'imc', 'imref', 'imbw']
statextract_light
0.1% saturation
measure
  35 par

In [7]:
# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

Modified:
    2023-01-11 18:36:22.553760
Pipeline steps:
    
1) Step: classifier
   Type: <class 'pyopia.classify.Classify'>
   Vars: {'model_path': 'keras_model.h5', 'class_labels': Index(['oil', 'other', 'bubble', 'faecal_pellets', 'copepod', 'diatom_chain',
       'oily_gas'],
      dtype='object'), 'model': <keras.engine.sequential.Sequential object at 0x00000270339DBF70>}
2) Step: create background
   Type: <class 'pyopia.background.CreateBackground'>
   Vars: {'bgfiles': ['raw_data\\D20201016T123936.323136.silc', 'raw_data\\D20201016T123936.510592.silc', 'raw_data\\D20201016T123936.682427.silc', 'raw_data\\D20201016T123936.869883.silc', 'raw_data\\D20201016T123937.057339.silc', 'raw_data\\D20201016T123937.244821.silc', 'raw_data\\D20201016T123937.432250.silc', 'raw_data\\D20201016T123937.604085.silc', 'raw_data\\D20201016T123937.791541.silc', 'raw_data\\D20201016T123937.972273.silc'], 'load_function': <function load_image at 0x0000027028382EE0>}
3) Step: load
   Type: <class 'py

In [8]:
import pyopia.statistics
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))
print('Number of raw images: ', pyopia.statistics.count_images_in_stats(stats))

stats header:  Index(['major_axis_length', 'minor_axis_length', 'equivalent_diameter',
       'solidity', 'minr', 'minc', 'maxr', 'maxc', 'probability_oil',
       'probability_other', 'probability_bubble', 'probability_faecal_pellets',
       'probability_copepod', 'probability_diatom_chain',
       'probability_oily_gas', 'export name', 'timestamp', 'saturation'],
      dtype='object')
Total number of particles:  668
Number of raw images:  20
